In [47]:
import json, os
import numpy as np
import cv2 
import matplotlib as plt

In [64]:
root_path        = r"."
masks_path       = r".\MASK"
json_files_path  = r".\JSON_LABELME"
json_files_list  = os.listdir(json_files_path)

In [61]:
mask_files_list  = []
for filename in json_files_list:
    mask_files_list.append(filename.replace("json", "png"))
print(mask_files_list)

['stuttgart_01_000000_003500_leftImg8bit.png', 'stuttgart_01_000000_003544_leftImg8bit.png', 'stuttgart_01_000000_003597_leftImg8bit.png', 'stuttgart_01_000000_003664_leftImg8bit.png', 'stuttgart_01_000000_003741_leftImg8bit.png', 'stuttgart_01_000000_003834_leftImg8bit.png', 'stuttgart_01_000000_003902_leftImg8bit.png', 'stuttgart_01_000000_004034_leftImg8bit.png', 'stuttgart_01_000000_004142_leftImg8bit.png', 'stuttgart_01_000000_004155_leftImg8bit.png', 'stuttgart_01_000000_004253_leftImg8bit.png', 'stuttgart_01_000000_004385_leftImg8bit.png']


In [ ]:
height = 1024
width = 2048

Label_Class = {
    "void" : 0,
    "DUONG_DI" : 255
}

for json_filename in json_files_list:
    # for each json file in the folder, open it
    with open(json_files_path + "\\" + json_filename) as json_file:
        # load data in json file
        json_data = json.load(json_file)
        # the data contains multi label, just focus to "shapes"
        labels_list = []
        points_list = []  # aka contour : đường viền :)
        for classes_of_object in json_data["shapes"]:
            # in the dict we have got - "shapes" also contain many labels
            # we just need "label" and "points" 
            # to make a mask
            my_label = classes_of_object["label"]
            my_points = classes_of_object["points"]
            # convert my_points from [[float, float], [float, float], ...] 
            # to [ (int, int), (int, int), ...] 
            # then to
            # [ [int int]
            #   [int int]
            #   ...     
            #   [int int] ]
            my_new_points = []
            # print(type(my_new_points))
            # print(my_new_points)
            for a_point in my_points:
                x = int(a_point[0])
                y = int(a_point[1])
                my_new_points.append((x,y))
            my_new_points = np.array(my_new_points)
            # print(type(my_new_points))
            # print(my_new_points)

            
            labels_list.append(my_label)
            points_list.append(my_new_points)
            # print(my_label)
            # print(my_new_points)
            # print(my_points)
        # make a mask image with zeros
        # np.zeros((height, width), dtype=np.uint8)
        mask_img = np.zeros((height, width), dtype=np.uint8)
        # mask the image
        for ith_label in range(len(labels_list)):
            for i in range(width):
                for j in range(height):
                    if cv2.pointPolygonTest(points_list[ith_label], (i, j), False) > 0:
                        mask_img[j, i] = Label_Class.get(labels_list[ith_label])

        # save mask image
        cv2.imwrite( masks_path + "\\" + json_filename.replace("json", "png"), mask_img)